In [1]:
#for progress bar to track time to complete for any given fxn
!pip install tqdm==4.36.1
!pip install pyrouge
!pip install pandas==1
!pip install joblib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52 kB 1.3 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.0
    Uninstalling tqdm-4.64.0:
      Successfully uninstalled tqdm-4.64.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.3.1 requires tqdm<5.0.0,>=4.38.0, but you have tqdm 4.36.1 which is incompatible.
panel 0.12.1 requires tqdm>=4.48.0, but you have tqdm 4.36.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 2.7 MB/s 
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191621 sha256=76ebc176a04b03cf8dac8f81773ff894922b6baa71f3371fec9eecd66dbfc792
  Stored in directory: /root/.cache/pip/w

In [2]:
import numpy as np
import pandas as pd
import os
import pickle
import re
import glob

#Loading Own Text Data Into Scikit
from sklearn.datasets import load_files
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
##from sklearn.feature_extraction.text import CountVectorizer
#to express the importance of a word to a document 
##from sklearn.feature_extraction.text import TfidfTransformer


from google.colab import drive
drive.mount('/content/drive')

import joblib
from drive.MyDrive.pyrouge import Rouge
from tqdm.auto import tqdm
tqdm.pandas()

Mounted at /content/drive


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# Load Word vector

In [3]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

#load glove file of 6 billion tokens and 50 dimensions
glove_file = datapath(os.path.abspath(r'/content/drive/MyDrive/project/glove.6B.50d.txt'))
#This function doesn’t creates file (only generate unique name). 
#Also, it may return different paths in consecutive calling.
#create temp file
tmp_file = get_tmpfile(os.path.abspath("test_word2vec.txt"))
#to convert the GloVe file format to the word2vec file format. 
converted_file = glove2word2vec(glove_file, tmp_file) 
#KeyedVectors.load_word2vec_format() function to load this model into memory
glove_model = KeyedVectors.load_word2vec_format(os.path.abspath("test_word2vec.txt"))

# vector representation of a word

In [4]:
glove_model["the"]

array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01],
      dtype=float32)

# **File Read**

In [5]:
#Stop word removal function
import gensim
all_stopwords = gensim.parsing.preprocessing.STOPWORDS

#from spacy.lang.en.stop_words import STOP_WORDS
def stop_word_remove(sentence):
  temp = [token for token in sentence.split() if token not in all_stopwords]
  return ' '.join(word for word in temp)

In [6]:
#load paper
def read_paper(path):
  
  f = open(path, 'r')
  text = str(f.read())
  f.close()
  return text

In [7]:
def bodyMain(text):
    text = re.sub("@&#", " ", text)
    text = re.sub("\n", " ", text)
    text = (text.encode('ascii', 'ignore')).decode("utf-8")
    

  # Extracting the highlights, body from the paper
    
    body_main = re.findall(r'INTRODUCTION(.*?)REFERENCES', text,  flags = re.I)[0] # check

    body = ' '.join(body_main.split())
    body = body.split(".")
    # body = [b for b in body[:-1]]
    return body

In [8]:
def abstract(text):
    # Removes unwanted characters, accounting for unicode characters
    text = re.sub("@&#", " ", text)
    text = re.sub("\n", " ", text)
    text = ' '.join(text.split())
    text = (text.encode('ascii', 'ignore')).decode("utf-8")
    
    abst = re.findall(r'.*(?:abstract)(.*?)introduction', text, flags=re.I)[0]
    return abst

In [9]:
pa = r'/content/drive/MyDrive/Parsed_Papers/Introduction/Introduction/S0168874X15000116.txt'
t2 = read_paper(pa)

In [10]:
t2

'@&#MAIN-TITLE@&#Optimal domain decomposition using Colliding Bodies Optimization and k-median method\n\n@&#HIGHLIGHTS@&#\n\n\n               \n               \n                  \n                     \n                        \n                           \n                           An efficient approach is presented for optimal domain decomposition of FEMs.\n\n\n                        \n                        \n                           \n                           A clique graph is used for transforming the connectivity of a finite element model into that of the graph.\n\n\n                        \n                        \n                           \n                           To decompose the nodes of a finite element model to k subdomains, k-median approach is employed.\n\n\n                        \n                        \n                           \n                           For optimal subdomain decomposition, a recently developed meta-heuristic algorithm, CBO, and P

In [ ]:
# extract(t2)

In [11]:
bodyMain(t2)

['Parallel processing has been widely applied to large-scale problem like the analysis of hydraulic systems, electrical networks and finite element meshes',
 ' The aim of a parallel algorithm is to decompose the given domain of system into subdomains, and analyze each subdomain by a processor',
 ' Given a number of available processors q, an arbitrary finite element model (FEM) is decomposed into q subdomains where formation of element matrices, assembly of global matrices, partial factorization of the stiffness matrix and state determination or evaluation of generalized stresses can be carried out independently of similar computations for the other subdomains, and hence can be performed in parallel',
 ' While the subdomains are processed in a parallel architecture, the time to complete a task will be the time to compute the longest subtask',
 ' An algorithm for optimizing the decomposed mesh can be considered as efficient if it yields subdomains that require an equal amount of executi

In [12]:
abstract(t2)

' An efficient approach is presented for optimal domain decomposition of finite element meshes in conjunction with k-median method and a meta-heuristic algorithm. A clique graph is used for transforming the connectivity of a finite element model (FEM) into that of the corresponding graph. Subsequently, to decompose the nodes of a graph or the meshes of a finite element model to k subdomains, k-median approach is employed. Then, for finding k medians of graph, a recently developed meta-heuristic algorithm, so-called Colliding Bodies Optimization (CBO), and standard Particle Swarm Optimization (PSO) are utilized. The performance of these methods is investigated through four FEMs to minimize the cost of k-median problem. A comparison of the numerical results using the CBO and PSO algorithms indicates that the CBO is capable of performing better decomposition using smaller or equal computational efforts. '

# **Clean Text**

In [13]:
#preprocessing
def process_paper(text):
    

  # Removes unwanted characters, accounting for unicode characters
    text = re.sub("@&#", " ", text)
    text = re.sub("\n", " ", text)
    text = ' '.join(text.split())
    text = (text.encode('ascii', 'ignore')).decode("utf-8")
    

  # Extracting the highlights, body from the paper
    highlights = re.findall(r'HIGHLIGHTS(.*?)KEYPHRASES', text,  flags = re.I)[0]
    abst = re.findall(r'ABSTRACT(.*?)INTRODUCTION', text, flags=re.I)[0]
    body_main = re.findall(r'INTRODUCTION(.*?)REFERENCES', text,  flags = re.I)[0]    
    
    
  # Making a copy of the body, lowercasing body text, removing punctuations & extra spaces
    dummy_body = body_main.lower()
    dummy_body = re.sub('[^\w\s\d\.]','',dummy_body)
    dummy_body = ' '.join(dummy_body.split())
    dummy_body = dummy_body.split(".")

  # Removing extra spaces from the body text, which will be preserved to produce summaries
  # And splitting into sentences
    body = ' '.join(body_main.split())
    body = body.split(".")

  # Removing sentences that are too short or too long, as they wouldn't make apt summary text
    for i,x in enumerate(dummy_body):
        if (len(x.split())) < 3 or (len(x.split())) > 15: 
            
            dummy_body.pop(i)
            body.pop(i)
            
  # Making a copy of the abstract, lowercasing abstract text, removing punctuations & extra spaces
    dummy_abst = abst.lower()
    dummy_abst = re.sub('[^\w\s\d\.]','',dummy_abst)
    dummy_abst = ' '.join(dummy_abst.split())
    dummy_abst = dummy_abst.split(".")
    
  # Removing extra spaces from the abstract text, which will be preserved to produce summaries And splitting into sentences
    ab = ' '.join(abst.split())
    ab = abst.split(".")
    
  # Removing sentences that are too short or too long, as they wouldn't make apt summary text
    for i,x in enumerate(dummy_abst):
        if (len(x.split())) < 3 or (len(x.split())) > 15: 
            
            dummy_abst.pop(i)
            ab.pop(i)

  # Making a copy of the highlights, lowercasing body text, removing punctuations & extra spaces
    dummy_highlights = highlights.lower()
    dummy_highlights = re.sub('[^\w\s\d]','',dummy_highlights)
    dummy_highlights = ' '.join(dummy_highlights.split())

  # Removing stop words from body & highlights
    body_copy = []
    for x in dummy_body:
      
        body_copy.append(stop_word_remove(x))
    highlight_copy = []
    for x in dummy_highlights.split():
        highlight_copy.append(stop_word_remove(x))
        
    # Removing stop words from abstarct
    abst_copy = []
    for x in dummy_abst:
      
        abst_copy.append(stop_word_remove(x))
        
    # Combing all of the abstract's sentences into one string    
    abst_copy = " ".join(sentence for sentence in abst_copy)
    abst_copy = " ".join(abst_copy.split())

  
  # Combing all of the highlights into one string    
    highlight_copy = " ".join(sentence for sentence in highlight_copy)
    highlight_copy = " ".join(highlight_copy.split())
    
    
    return body_main, body_copy, highlights, highlight_copy, abst , abst_copy

In [14]:
body_main, body_copy, highlights, highlight_copy, abst , abst_copy = process_paper(t2)

In [15]:
body_main

' Parallel processing has been widely applied to large-scale problem like the analysis of hydraulic systems, electrical networks and finite element meshes. The aim of a parallel algorithm is to decompose the given domain of system into subdomains, and analyze each subdomain by a processor. Given a number of available processors q, an arbitrary finite element model (FEM) is decomposed into q subdomains where formation of element matrices, assembly of global matrices, partial factorization of the stiffness matrix and state determination or evaluation of generalized stresses can be carried out independently of similar computations for the other subdomains, and hence can be performed in parallel. While the subdomains are processed in a parallel architecture, the time to complete a task will be the time to compute the longest subtask. An algorithm for optimizing the decomposed mesh can be considered as efficient if it yields subdomains that require an equal amount of execution time. Some al

In [16]:
body_copy

['aim parallel algorithm decompose given domain subdomains analyze subdomain processor',
 'subdomains processed parallel architecture time complete task time compute longest subtask',
 'algorithms developed problem finding optimal domain finite elements methods 16 review papers topic available 78',
 'algorithms works 912',
 'approximate algorithms developed finding medians graph',
 'method graph associated connectivity property considered finite element',
 'recently metaheuristics algorithms genetic algorithms 1314 bionomic approaches 15 ant colony 1617 developed order obtain solutions kmedian problem',
 'algorithm considered multiagent method agent colliding body cb',
 'collision occurs cb moves new position according new velocity',
 'algorithm utilizes simple formulation requires internal parameter tuning',
 'standard pso algorithm utilized comparison study',
 'simple graph g defined set ng nodes set edges relation incidence associate distinct nodes edge known ends',
 'edge called in

In [17]:
highlights

' An efficient approach is presented for optimal domain decomposition of FEMs. A clique graph is used for transforming the connectivity of a finite element model into that of the graph. To decompose the nodes of a finite element model to k subdomains, k-median approach is employed. For optimal subdomain decomposition, a recently developed meta-heuristic algorithm, CBO, and PSO are utilized. The performance of the methods is investigated through four FEMs. A comparison of the results of CBO and PSO algorithms is performed. '

In [18]:
highlight_copy

'efficient approach presented optimal domain decomposition fems clique graph transforming connectivity finite element model graph decompose nodes finite element model k subdomains kmedian approach employed optimal subdomain decomposition recently developed metaheuristic algorithm cbo pso utilized performance methods investigated fems comparison results cbo pso algorithms performed'

In [19]:
abst

' An efficient approach is presented for optimal domain decomposition of finite element meshes in conjunction with k-median method and a meta-heuristic algorithm. A clique graph is used for transforming the connectivity of a finite element model (FEM) into that of the corresponding graph. Subsequently, to decompose the nodes of a graph or the meshes of a finite element model to k subdomains, k-median approach is employed. Then, for finding k medians of graph, a recently developed meta-heuristic algorithm, so-called Colliding Bodies Optimization (CBO), and standard Particle Swarm Optimization (PSO) are utilized. The performance of these methods is investigated through four FEMs to minimize the cost of k-median problem. A comparison of the numerical results using the CBO and PSO algorithms indicates that the CBO is capable of performing better decomposition using smaller or equal computational efforts. '

In [20]:
abst_copy

'clique graph transforming connectivity finite element model fem corresponding graph finding k medians graph recently developed metaheuristic algorithm socalled colliding bodies optimization cbo standard particle swarm optimization pso utilized comparison numerical results cbo pso algorithms indicates cbo capable performing better decomposition smaller equal computational efforts'

# **Word Embedding**

In [21]:
gloveFile= '/content/drive/MyDrive/project/glove.6B.50d.txt'
dim = 50
def loadGloveModel(gloveFile):
    word_embeddings = {}
    f = open(gloveFile, encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()
    return word_embeddings
 
word_embeddings = loadGloveModel(gloveFile)

# **Document Vector**

In [22]:
def create_sentence_vectors(body_copy):
  sentence_vectors = []
  for i in body_copy:
      if len(i) != 0:
          v = sum([word_embeddings.get(w, np.zeros((dim,))) for w in i.split()])/(len(i.split())+0.001)
      else:
          v = np.zeros((dim,))
      sentence_vectors.append(v)
  return sentence_vectors

In [23]:
len(create_sentence_vectors(body_copy))

107

# **Document Score**

`NOTE` expit is nothing but logistic sigmoid.

- Document Score is Calculated using glove embeddings.

- The function will calculate sentence score 

In [24]:
from scipy.special import expit

def document_score(body_copy, abst_copy):
    
  # Getting word vectors for the body
    body_vectors = []
    for sent in body_copy:
        sent_vec = []
        #split word by space
        for word in sent.split():
            try:
                #attach it's embedding if present
                sent_vec.append(glove_model[word])
          # If the word vector isn't there in the model
          # then use the vector of the word "Visual"
            except:
                sent_vec.append(glove_model["visual"])
        #append all vectors is body_vector list
        body_vectors.append(sent_vec)

  # Getting word vectors for the highlights
    abst_vectors = []
    for word in abst_copy.split():
        try:
            abst_vectors.append(glove_model[word])
        except:
            abst_vectors.append(glove_model["visual"])

  # Finding the rouge score for each sentence by counting the # of common words
  # & dividing by length of sentence
  #highlights are user generated summaries
    doc_score = []
    for sent in body_vectors:
        sent_score = 0
        for word in sent:
            for w in abst_vectors:
                if (word == w).all():
                    sent_score+=1
        try: 
            #sigmoid fxn: 1/(1+exp(-x))
            doc_score.append(expit(sent_score/len(sent)))
        except:
            doc_score.append(0)
    return doc_score

In [25]:
doc_score = document_score(body_copy, abst_copy)

In [26]:
doc_score

[0.52497918747894,
 0.5,
 0.5662743941954392,
 0.5825702064623147,
 0.7625419716560975,
 0.679178699175393,
 0.555328055262369,
 0.5621765008857981,
 0.5,
 0.5312093733737563,
 0.7310585786300049,
 0.5440035103251184,
 0.5,
 0.517234548877195,
 0.5,
 0.5,
 0.5379702337831526,
 0.5356536708339716,
 0.5,
 0.5293778879892553,
 0.574442516811659,
 0.5,
 0.5356536708339716,
 0.5,
 0.5356536708339716,
 0.5,
 0.5,
 0.5262915173930184,
 0.5,
 0.5277492350545132,
 0.5393918791114867,
 0.5,
 0.5677623235383435,
 0.5926665999540697,
 0.6390927451628512,
 0.6607563687658172,
 0.5,
 0.5941027850878001,
 0.520821285372743,
 0.6055324872205857,
 0.5356536708339716,
 0.5227116332606392,
 0.52497918747894,
 0,
 0.5,
 0.5299640517645717,
 0.5,
 0.5,
 0.5138853177460049,
 0.5,
 0,
 0.5,
 0.5621765008857981,
 0.5,
 0.5,
 0,
 0.5,
 0,
 0.5,
 0.5356536708339716,
 0.5,
 0.5,
 0,
 0.5,
 0,
 0.5,
 0.5453297388885201,
 0.5873306243819909,
 0.7310585786300049,
 0.5621765008857981,
 0.5,
 0.6354235592583607,
 0.5

In [27]:
len(doc_score)

107

# **Pipeline**

In [28]:
# Function to create datasets
def create_data(path):
    text = read_paper(path)
    
    body_main, body_copy, highlights, highlight_copy, abst , abst_copy = process_paper(text)
    
    
    doc_score = document_score(body_copy, abst_copy)
    sent_vectors = create_sentence_vectors(body_copy) 

    x = sent_vectors    
    x = pd.DataFrame(x)
    # print(x.head())
    y = pd.DataFrame(doc_score)
    # print(y.head())
    return x, y

# **Train-Test Split**

In [ ]:
file_name = [i.split('/')[-1].replace('.txt', '') for i in glob.glob("/content/drive/MyDrive/Parsed_Papers/Introduction/Introduction/*.txt")]
filenames = glob.glob(f"/content/drive/MyDrive/Parsed_Papers/Introduction/Introduction/*.txt")

train_files = filenames[:160]
test_files = filenames[161:201]

import shutil, os

if not os.path.exists('/content/drive/MyDrive/Parsed_Papers/Introduction/Train_Files/'):
  os.makedirs('/content/drive/MyDrive/Parsed_Papers/Introduction/Train_Files/')
if not os.path.exists('/content/drive/MyDrive/Parsed_Papers/Introduction/Test_Files/'):
  os.makedirs('/content/drive/MyDrive/Parsed_Papers/Introduction/Test_Files/')


Train_Folder = '/content/drive/MyDrive/Parsed_Papers/Introduction/Train_Files/'
Test_Folder = '/content/drive/MyDrive/Parsed_Papers/Introduction/Test_Files/'

for train_file in train_files:
  shutil.copy(train_file, Train_Folder)

for test_file in test_files:
  shutil.copy(test_file, Test_Folder)

In [29]:
test = glob.glob(f"/content/drive/MyDrive/Parsed_Papers/Introduction/Test_Files/*.txt")

In [30]:
len(test)

40

# **Train**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor

Model = SGDRegressor()
import warnings
warnings.filterwarnings("ignore")


train = glob.glob(f"/content/drive/MyDrive/Parsed_Papers/Introduction/Train_Files/*.txt")
for i,path in enumerate(tqdm(train[:])):
    file_name1 = path.split('/')[-1].replace('.txt', '')
    print("file_name: ",file_name1)
    X_train, y_train= create_data(path)
    Model.fit(X_train,y_train)

file_name:  S0167739X13001349
         0         1         2         3         4         5         6   \
0  0.293603 -0.066804  0.357167  0.147317  0.120867  0.129181 -0.029661   
1  0.069505  0.015534  0.120104 -0.036709  0.062860 -0.123455 -0.440143   
2  0.738480 -0.256125  0.611636  0.097627  0.079180 -0.107557 -0.348986   
3  0.296634 -0.357480 -0.234086  0.057754  0.032576  0.145484  0.238877   
4  0.652066 -0.010899  0.269191 -0.094421 -0.066686  0.228340 -0.314169   

         7         8         9   ...        40        41        42        43  \
0 -0.385756  0.325513  0.169786  ...  0.118581 -0.102048 -0.106740  0.566403   
1 -0.312274  0.209295  0.506990  ... -0.420321 -0.204763  0.025999 -0.143469   
2  0.022283 -0.083579  0.222198  ... -0.034359  0.120749  0.100803  0.670636   
3 -0.446279 -0.142177  0.612953  ...  0.188204 -0.367696  0.062469  0.608549   
4 -0.688653 -0.010605  0.264301  ... -0.036406 -0.174670  0.337954  0.368502   

         44        45        46       

# **save model**

In [ ]:
import joblib
joblib.dump(Model,"/content/drive/MyDrive/project/Text_summarization_RandomForest_Regressor_model_160.pkl")


['/content/drive/MyDrive/project/Text_summarization_RandomForest_Regressor_model_160.pkl']

In [32]:
model=joblib.load("/content/drive/MyDrive/project/Text_summarization_sgdregressor_model_160.pkl")

# **Test**
In case of linear regression we have to call the function `flatten(l)` since y_pred is a multidimentional list and we have to convert it to 1D.

In [33]:
def bodyMain(path):
    text = read_paper(path)
    text = re.sub("@&#", " ", text)
    text = re.sub("\n", " ", text)
    text = (text.encode('ascii', 'ignore')).decode("utf-8")
    

  # Extracting the highlights, body from the paper
    
    body_main = re.findall(r'INTRODUCTION(.*?)REFERENCES', text,  flags = re.I)[0]

    body = ' '.join(body_main.split())
    body = body.split(".")
    return body


In [34]:
def flatten(l):
    """ Flatten a list of lists"""
    return [item for sublist in l for item in sublist]

In [42]:
test = glob.glob(f"/content/drive/MyDrive/Parsed_Papers/Introduction/Test_Files/*.txt")
col_names = ['FileName','Abstract','Summary','Precision','Recall','F1_score']

result = pd.DataFrame(columns=col_names,dtype=object)

for i,path in enumerate(tqdm(test[:])):
    print('i',i)
    print("path",path)
    file_name = path.split('/')[-1].replace('.txt', '')
    print("file_name",file_name)
    X_test, y_test= create_data(path)

    y_pred = model.predict(X_test)
    # y_pred = flatten(y_pred) # use only for linear regression otherwise command it
    
    #actual score
    y_test = y_test.rename(columns = {y_test.columns[0]:'Actual_score'})
    actual = y_test['Actual_score'].tolist()

    #top 6 sentence as summary
    pred = pd.Series(y_pred)
    predicted = pred.tolist()
    idx = pred.nlargest(6)
    idx = idx.index.values.tolist()

    # df_actual = pd.DataFrame({'FileName':file_name,'Actual_Score':[actual]},dtype = object)
    # df_pred = pd.DataFrame({'FileName':file_name,'Predicted_score':[predicted]},dtype = object)


    body=bodyMain(path)

    summary = []

    for x in range(6):
        summary.append(body[idx[x]])

    s=""
    for i in range(len(summary)):
      s += summary[i]+'.'



    text = read_paper(path)
    abstractL = abstract(text)
    print("abstract: ",abstractL)

    # precision recall f score
    r = Rouge()

    [precision, recall, f_score] = r.rouge_l([abstractL], [s])

    df = pd.DataFrame(columns=col_names,dtype=object)

    df.loc[i, ['FileName']] = file_name
    df.loc[i,['Abstract']] = abstractL
    df.loc[i,['Summary']] =  s
    df.loc[i, ['Precision']] = precision
    df.loc[i, ['Recall']] = recall
    df.loc[i, ['F1_score']] = f_score

    # df = df.merge(df_actual, on = 'FileName')
    # df = df.merge(df_pred, on = 'FileName')

    result = pd.concat([result, df], axis = 0)

i 0
path /content/drive/MyDrive/Parsed_Papers/Introduction/Test_Files/S0168874X14001413.txt
file_name S0168874X14001413
abstract:   The topic of the paper is related to the macroscopic modelling of riveted assemblies for full-scale aircraft structures subjected to crash/impact loadings. Super-elements have been developed to model the rivet and its rupture. However, stress concentrations in riveted assemblies can also initiate cracks from hole boundaries that propagate to the next ones and can lead to the catastrophic loss of the airplane by dismantlement of structural parts. An hybrid-Trefftz perforated super-element has been previously developed by the authors. However, the hole boundary of this super-element is a traction-free analytical boundary, preventing any connection with rivet elements. As a necessary first step to make the connection with the rivet possible, it is proposed to formulate additional nodes on the hole boundary of the perforated super-element. However, increasing 

In [43]:
res = result.copy()
res.reset_index(inplace = True)

In [44]:
res.drop(['index'], axis = 1,inplace = True)

In [45]:
res.head(2)

,FileName,Abstract,Summary,Precision,Recall,F1_score
0,S0168874X14001413,The topic of the paper is related to the macr...,The truncation order of the series in the KM ...,0.329167,0.296065,0.311741
1,S0168874X14001425,Multi-scale finite element (FE) modeling and ...,"In surfacing coupling, there is no overlappin...",0.346032,0.592391,0.436875


In [ ]:
res.to_csv('/content/drive/MyDrive/project/result/SGD_Regressor.csv')